# Corona Virus in the US

In [149]:
# data wrangle
import pandas as pd
import numpy as np

import json

### Aquire - deaths by county

In [153]:
df = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv")

In [154]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19713 entries, 0 to 19712
Data columns (total 6 columns):
date      19713 non-null object
county    19713 non-null object
state     19713 non-null object
fips      19407 non-null float64
cases     19713 non-null int64
deaths    19713 non-null int64
dtypes: float64(1), int64(2), object(3)
memory usage: 924.2+ KB


In [155]:
df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


### Prep

In [156]:
df.isnull().sum()

date        0
county      0
state       0
fips      306
cases       0
deaths      0
dtype: int64

In [157]:
df [df.fips.isnull()]

,date,county,state,fips,cases,deaths
416,2020-03-01,New York City,New York,NaN,1,0
418,2020-03-01,Unknown,Rhode Island,NaN,2,0
448,2020-03-02,New York City,New York,NaN,1,0
450,2020-03-02,Unknown,Rhode Island,NaN,2,0
482,2020-03-03,New York City,New York,NaN,2,0
...,...,...,...,...,...,...
19219,2020-03-29,Unknown,Rhode Island,NaN,106,3
19356,2020-03-29,Unknown,Tennessee,NaN,190,4
19495,2020-03-29,Unknown,Utah,NaN,1,1
19509,2020-03-29,Unknown,Vermont,NaN,23,5


In [158]:
df [df.fips.isnull()].county.value_counts()

Unknown          268
New York City     29
Kansas City        9
Name: county, dtype: int64

### Explore

In [159]:
states = df.groupby(df.state).sum()[["cases","deaths"]]

In [160]:
states["death_rate"] = states.deaths / states.cases * 100

In [162]:
states.index.name = None

In [163]:
states.sort_values("death_rate", ascending=False)

,cases,deaths,death_rate
Washington,35218,1931,5.482992
Vermont,1409,67,4.755145
South Dakota,504,20,3.968254
Louisiana,19666,712,3.620462
Puerto Rico,582,21,3.608247
Georgia,15008,471,3.138326
Oklahoma,2022,61,3.016815
Oregon,3326,95,2.856284
Indiana,6130,160,2.610114
Kansas,1599,41,2.564103


#### Add population data

State population data obtained from https://worldpopulationreview.com/states/

In [166]:
with open('data.json') as f:
    pop = json.load(f)

In [180]:
pop.keys()

dict_keys(['data'])

In [174]:
pd.DataFrame(pop['data'])

,rank,State,Pop,Growth,Pop2018,Pop2010,growthSince2010,Percent,density
0,1,California,39937489,0.0096,39557045,37320903,0.0701,0.1194,256.3727
1,2,Texas,29472295,0.0268,28701845,25242679,0.1676,0.0881,112.8204
2,3,Florida,21992985,0.0326,21299325,18845785,0.1670,0.0658,410.1256
3,4,New York,19440469,-0.0052,19542209,19400080,0.0021,0.0581,412.5211
4,5,Pennsylvania,12820878,0.0011,12807060,12711158,0.0086,0.0383,286.5449
5,6,Illinois,12659682,-0.0064,12741080,12840762,-0.0141,0.0379,228.0243
6,7,Ohio,11747694,0.0050,11689442,11539327,0.0181,0.0351,287.5038
7,8,Georgia,10736059,0.0206,10519475,9711810,0.1055,0.0321,186.6719
8,9,North Carolina,10611862,0.0220,10383620,9574293,0.1084,0.0317,218.2702
9,10,Michigan,10045029,0.0049,9995915,9877535,0.0170,0.0300,177.6655
